In [13]:
import util
from test_framework.address import program_to_witness
from test_framework.key import ECPubKey, generate_bip340_key_pair, ECKey
from test_framework.messages import CTxInWitness, ser_string
from test_framework.script import tagged_hash, Tapbranch, TapTree, TapLeaf, CScript, TaprootSignatureHash, OP_CHECKSIG, SIGHASH_ALL_TAPROOT

In [14]:
privkey_staker, pubkey_staker = generate_bip340_key_pair()
privkey_fp, pubkey_fp = generate_bip340_key_pair()
privkey_c1, pubkey_c1 = generate_bip340_key_pair()
privkey_c2, pubkey_c2 = generate_bip340_key_pair()
privkey_c3, pubkey_c3 = generate_bip340_key_pair()

internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0')) # babylon internal public key

50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0

In [15]:
# Generate timelock tapscript
delay = 20
tapleaf_timelock = TapLeaf().construct_pk_delay(pubkey_staker, delay=delay)
print("Tapscript descriptor:", tapleaf_timelock.desc, "\n")

print("Tapscript operations:")
for op in tapleaf_timelock.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in tapleaf_timelock.sat:
    print("{}, {}".format(element, value.hex()))

Tapscript descriptor: ts(pk_delay(23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890,20)) 

Tapscript operations:
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
OP_CHECKSIG
OP_VERIFY
14
OP_CHECKSEQUENCEVERIFY

Satisfying witness elements:
sig, 23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890


In [16]:
# Generate unbonding tapscript
tapleaf_unbonding = TapLeaf().construct_pk_csa(pubkey_staker, 2, [pubkey_c1, pubkey_c2, pubkey_c3])
# tapleaf_unbonding = TapLeaf().construct_csa(2, [pubkey_c1, pubkey_c2, pubkey_c3])
print("Tapscript descriptor:", tapleaf_unbonding.desc, "\n")

print("Tapscript operations:")
for op in tapleaf_unbonding.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in tapleaf_unbonding.sat:
    print("{}, {}".format(element, value.hex()))

Tapscript descriptor: ts(pk_csa(23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890,2,195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85,9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf,802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59)) 

Tapscript operations:
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
OP_CHECKSIG
OP_VERIFY
195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85
OP_CHECKSIG
9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf
OP_CHECKSIGADD
802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59
OP_CHECKSIGADD
2
OP_NUMEQUAL

Satisfying witness elements:
sig, 802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59
sig, 9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf
sig, 195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85
sig, 23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890


In [17]:
# Generate slashing tapscript
tapleaf_slashing = TapLeaf().construct_pk_pk_csa(pubkey_staker, pubkey_fp, 2, [pubkey_c1, pubkey_c2, pubkey_c3])
print("Tapscript descriptor:", tapleaf_unbonding.desc, "\n")

print("Tapscript operations:")
for op in tapleaf_slashing.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in tapleaf_slashing.sat:
    print("{}, {}".format(element, value.hex()))

Tapscript descriptor: ts(pk_csa(23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890,2,195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85,9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf,802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59)) 

Tapscript operations:
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
OP_CHECKSIG
OP_VERIFY
18c0b351218338e7d8f43f2ce03b385270a9c81b4155e3a6c5c7b801f51dbab9
OP_CHECKSIG
OP_VERIFY
195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85
OP_CHECKSIG
9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf
OP_CHECKSIGADD
802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59
OP_CHECKSIGADD
2
OP_NUMEQUAL

Satisfying witness elements:
sig, 802a11d7d9ed8e0df683fe96315f963f49f5f0775062be70204c955234badc59
sig, 9e7ef674994a4c342dd54c278e9e2f89c1dcbb75eca6799da40bcf2d1ae8abcf
sig, 195a01a6b1a55f6c54f6ec5b82db7a4f579f230bc13e5c026ef3b49aa8ee6b85
sig, 18c0b35121

In [18]:
# Construct taptree nodes.
tapbranchAB = Tapbranch(tapleaf_unbonding, tapleaf_timelock)
tapbranchABC = Tapbranch(tapleaf_slashing, tapbranchAB)

# Construct the taptree.
taptree = TapTree(key=internal_pubkey, root=tapbranchABC)

segwit_v1_script, tweak, control_map = taptree.construct()

taproot_pubkey = internal_pubkey.tweak_add(tweak) 
program = taproot_pubkey.get_bytes()
address = program_to_witness(1, program)
print("Address: {}".format(address))

Address: bcrt1pa6tugfd92eqnwk0lrhx8tczlu67frzg8ss6x5x6y4z95wrdm75xqf60pc0


### ---- Spend

In [19]:
# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

# Generate coins and create an output
tx = node.generate_and_send_coins(address)
print("Transaction {}, output 0\nsent to {}\n".format(tx.hash, address))

2024-06-06T11:16:19.513000Z TestFramework./var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_t2_mo_8u (INFO): Initializing test directory /var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_t2_mo_8u

Client version is /Satoshi:26.99.0(testnode0)/

Balance: 50.00000000

Transaction bd05e0c122e134439e52c9d00bf3ad66d04d0a3762523973cfaa506f8f07a6e4, output 0
sent to bcrt1pa6tugfd92eqnwk0lrhx8tczlu67frzg8ss6x5x6y4z95wrdm75xqf60pc0



In [20]:
# Create a spending transaction
nSequence = delay
spending_tx = test.create_spending_transaction(tx.hash, version=2, nSequence=nSequence)

print("Spending transaction:\n{}".format(spending_tx))

Spending transaction:
CTransaction(nVersion=2 vin=[CTxIn(prevout=COutPoint(hash=bd05e0c122e134439e52c9d00bf3ad66d04d0a3762523973cfaa506f8f07a6e4 n=0) scriptSig= nSequence=20)] vout=[CTxOut(nValue=0.50000000 scriptPubKey=00143823cb4d158b933a2be7165e76d771c120baf1d6)] wit=CTxWitness() nLockTime=0)


### Spent by timelock

In [21]:
# Generate the taproot signature hash for signing
sighash = TaprootSignatureHash(
    spending_tx,
    [tx.vout[0]],
    SIGHASH_ALL_TAPROOT,
    input_index=0,
    scriptpath=True,
    script=tapleaf_timelock.script,
)

signature_staker = privkey_staker.sign_schnorr(sighash)
print("Signature for TapLeaf: {}\n".format(signature_staker.hex()))

# Add witness to transaction
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness_elements = [signature_staker, tapleaf_timelock.script, control_map[tapleaf_timelock.script]]
spending_tx.wit.vtxinwit.append(CTxInWitness(witness_elements))

# Test mempool acceptance
assert not node.test_transaction(spending_tx)
node.generate(delay)
assert node.test_transaction(spending_tx)
print("Success!")

Signature for TapLeaf: 7e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a679c9634a0845e33e5ab4f9be93d31359499952b27ca26e08fe19651f7a7dd4aa4

{'txid': 'd71744d5d771d4e24fa77b0d794355e79c0351bdb21a57be706aa51f8aefaae6', 'wtxid': '6c3f7bc1618cd909d872fa4c71d2b3e5b0acae41ddb2eab5541469df8fa6a3e9', 'allowed': False, 'reject-reason': 'non-BIP68-final'}
{'txid': 'd71744d5d771d4e24fa77b0d794355e79c0351bdb21a57be706aa51f8aefaae6', 'wtxid': '6c3f7bc1618cd909d872fa4c71d2b3e5b0acae41ddb2eab5541469df8fa6a3e9', 'allowed': True, 'vsize': 134, 'fees': {'base': Decimal('0.50000000'), 'effective-feerate': Decimal('3.73134328'), 'effective-includes': ['6c3f7bc1618cd909d872fa4c71d2b3e5b0acae41ddb2eab5541469df8fa6a3e9']}}
Success!


### Spent by unbonding

In [11]:
# Generate the taproot signature hash for signing
sighash_unbonding = TaprootSignatureHash(
    spending_tx,
    [tx.vout[0]],
    SIGHASH_ALL_TAPROOT,
    input_index=0,
    scriptpath=True,
    script=tapleaf_unbonding.script,
)

signature_staker = privkey_staker.sign_schnorr(sighash_unbonding)
signature_c1 = privkey_c1.sign_schnorr(sighash_unbonding)
signature_c2 = privkey_c2.sign_schnorr(sighash_unbonding)
signature_c3 = privkey_c3.sign_schnorr(sighash_unbonding)
print("Signature for TapLeaf: {}".format(signature_staker.hex()))
print("Signature for TapLeaf: {}".format(signature_c1.hex()))
print("Signature for TapLeaf: {}".format(signature_c2.hex()))
print("Signature for TapLeaf: {}\n".format(signature_c3.hex()))

# Add witness to transaction
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness_elements = [
    signature_c3,
    signature_c2,
    b'',
    signature_staker,
    tapleaf_unbonding.script,
    control_map[tapleaf_unbonding.script],
]
spending_tx.wit.vtxinwit.append(CTxInWitness(witness_elements))

# Test mempool acceptance
# assert not node.test_transaction(spending_tx)
# node.generate(delay)
assert node.test_transaction(spending_tx)
print("Success!")

Signature for TapLeaf: e0164caba1bcee36d4373452b1bdf729ba167598d5cdd4a0b6f146e690c4c2e352565852e5e2b0b2ff0d197852365d86ce0d2fc92d8fd1158117689e95c03805
Signature for TapLeaf: b35dafb49491150c1a49144999579c016ee5e13dc2314e31e3e30e1d3db99b0b0c08bf893831bbb6d13064e6887ea129b8b85cb3a5f6ae11a49206bf103cda13
Signature for TapLeaf: 0dba987a0abae11d0e186f81ce7edb85b17c63cead9b4ebcb2de3e6b303f2fa6fe5568932bdcfc7ae3c52e70cf2738be9278afdf3d06b6979e2857e615131f5c
Signature for TapLeaf: 66925ddb0ed7d3cb341bd138d9aa9e1aba16764ce2e18292d217407ad939d034102a56d5f86a91ca685293580b7fadd0f7b9ebe24974371cce115cff4c90847a

{'txid': '884da791e731203fe4070259c9004c011a52a78fa10570c9958ef7691cc18ba1', 'wtxid': '58154302e5498ae03dc2f4caeafe9637d9e85c61bf64f5698faa9b72856ba2b4', 'allowed': True, 'vsize': 209, 'fees': {'base': Decimal('0.50000000'), 'effective-feerate': Decimal('2.39234449'), 'effective-includes': ['58154302e5498ae03dc2f4caeafe9637d9e85c61bf64f5698faa9b72856ba2b4']}}
Success!


### Spent by slashing

In [9]:
# Generate the taproot signature hash for signing
sighash_slashing = TaprootSignatureHash(
    spending_tx,
    [tx.vout[0]],
    SIGHASH_ALL_TAPROOT,
    input_index=0,
    scriptpath=True,
    script=tapleaf_slashing.script,
)

signature_staker = privkey_staker.sign_schnorr(sighash_slashing)
signature_fp = privkey_fp.sign_schnorr(sighash_slashing)
signature_c1 = privkey_c1.sign_schnorr(sighash_slashing)
signature_c2 = privkey_c2.sign_schnorr(sighash_slashing)
signature_c3 = privkey_c3.sign_schnorr(sighash_slashing)
print("Signature for TapLeaf: {}".format(signature_staker.hex()))
print("Signature for TapLeaf: {}".format(signature_fp.hex()))
print("Signature for TapLeaf: {}".format(signature_c1.hex()))
print("Signature for TapLeaf: {}".format(signature_c2.hex()))
print("Signature for TapLeaf: {}\n".format(signature_c3.hex()))

# Add witness to transaction
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness_elements = [
    signature_c3,
    signature_c2,
    b'',
    signature_fp,
    signature_staker,
    tapleaf_slashing.script,
    control_map[tapleaf_slashing.script],
]
spending_tx.wit.vtxinwit.append(CTxInWitness(witness_elements))

# Test mempool acceptance
# assert not node.test_transaction(spending_tx)
# node.generate(delay)
assert node.test_transaction(spending_tx)
print("Success!")

Signature for TapLeaf: ea657f0bf39ef6a510479f42dbf843a300dd20b5bca570d1f43484276e519534d3af192ed4ceadda271011ac1eddf19c09225e4f96a4a210fd4be1dd2288a676
Signature for TapLeaf: 805c4d260c18382744cba2a6fdb5035d45b6ef2c5e8849abf7bf72e1c8ed32348c0c21d1131c08bf12ea0b40533864a21fc72ad082854945e89696484f11829f
Signature for TapLeaf: 29c85fade761d4f871a5a1e01e6fad0cb9401588ad4656165179483beead39d38591df3ffee88ada31f1978ab07a50dbe19d9483c1ade160114abef521b99015
Signature for TapLeaf: 2685ccf7c91979155908534d275b6be71eac2a73795a6d322b2dca064339ff095ee2788dcc196586087cc3fb3ed25f641ad14c179c233cfb04967417efebff3a
Signature for TapLeaf: ab2ca5e12192c1b93d4dc36c41d277b39fe0623057156d47dee3d87d934220ce221a25e9512cdc68e4b69136a960b26c370619d576e732efe5c9f1003c613a4b

{'txid': '884da791e731203fe4070259c9004c011a52a78fa10570c9958ef7691cc18ba1', 'wtxid': '58154302e5498ae03dc2f4caeafe9637d9e85c61bf64f5698faa9b72856ba2b4', 'allowed': True, 'vsize': 209, 'fees': {'base': Decimal('0.50000000'), 'effective-fee

In [22]:
# import subprocess

print(f"btcdeb --txin={tx.serialize().hex()} --tx={spending_tx.serialize().hex()}")

# subprocess.call(["btcdeb", f"--txin={txin}", f"--tx={tx}"], shell=True) 

btcdeb --txin=0200000000010174f4d667a9eabdf8957c425ddc69b7650f7195573c1de0dc9e04aadaa66169000000000000fdffffff0100e1f50500000000225120ee97c425a556413759ff1dcc75e05fe6bc91890784346a1b44a88b470dbbf50c0247304402200523c8109c8fb2fad88d8957ade2a4fdf0d9d679da8b0c7ca648721096c472f0022017d6751c277d4f3d3b3926278c308e21a8018c890021336c21553296d8380679012102d0121f14d9d6bffa3789a5c08c47f0ad3cd6009081d23fc3d7609b3c20929d3f00000000 --tx=02000000000101e4a6078f6f50aacf73395262370a4dd066adf30bd0c9529e4334e122c1e005bd0000000000140000000180f0fa02000000001600143823cb4d158b933a2be7165e76d771c120baf1d603407e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a679c9634a0845e33e5ab4f9be93d31359499952b27ca26e08fe19651f7a7dd4aa4262023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b261c150929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661df4464b2d00000000

In [12]:
# Shutdown
test.shutdown()

2024-06-06T11:13:50.139000Z TestFramework./var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_15s8dxyt (INFO): Stopping nodes
2024-06-06T11:13:50.306000Z TestFramework./var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_15s8dxyt (INFO): Cleaning up /var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_15s8dxyt on exit
2024-06-06T11:13:50.307000Z TestFramework./var/folders/b_/p0cr2yhn6kn1kc2rrcr1564m0000gn/T/bitcoin_func_test_15s8dxyt (INFO): Tests successful


### timelock

btcdeb --txin=02000000000101d8b958b696fe4a763b23f91811a7108fe979256d823bcaff6bcb3ae6de6a0a7a0000000000fdffffff0100e1f5050000000022512065a0580be484f39ab366f7e8a3f8186db9efaf568a8d7c1d27c72eab26adbb1d0247304402200a55bba78310bc2c8f24247138383d63acaba2f183c84c82dfa032ac11e2cfc602201689aab21c0e5c03d0d03445bc9e56e7e013b6eb68d23e441adb7c80b81283c6012102535b41f53240cc80a211367b8a0f9e698072008c3491c690845ac36ad71ff74800000000 --tx=020000000001015e3e704d96445f121903f038364b9d238bb3119740f7a6e7746aeb163a95a0f40000000000140000000180f0fa020000000016001449d241a0ab52a7deaa79e48ba77d8150b13092f50340fc014bd848cfd2db20e24082be0409821f76812ea069ad24958e91f052ccafdde81bc457ca5b5e8cf5cfcb400578cad1c90b0c2a61831a5effb71696ac6fbe292620fcbfe987672b759ed2699be15980e3ff2f4ea8eb4fa3c59c9a3f2205bf646564ac690114b221c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000

```bash
$ btcdeb --txin=0200000000010174f4d667a9eabdf8957c425ddc69b7650f7195573c1de0dc9e04aadaa66169000000000000fdffffff0100e1f50500000000225120ee97c425a556413759ff1dcc75e05fe6bc91890784346a1b44a88b470dbbf50c0247304402200523c8109c8fb2fad88d8957ade2a4fdf0d9d679da8b0c7ca648721096c472f0022017d6751c277d4f3d3b3926278c308e21a8018c890021336c21553296d8380679012102d0121f14d9d6bffa3789a5c08c47f0ad3cd6009081d23fc3d7609b3c20929d3f00000000 --tx=02000000000101e4a6078f6f50aacf73395262370a4dd066adf30bd0c9529e4334e122c1e005bd0000000000140000000180f0fa02000000001600143823cb4d158b933a2be7165e76d771c120baf1d603407e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a679c9634a0845e33e5ab4f9be93d31359499952b27ca26e08fe19651f7a7dd4aa4262023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b261c150929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661df4464b2d00000000
btcdeb 5.0.24 -- type `btcdeb -h` for start up options
LOG: signing segwit taproot
notice: btcdeb has gotten quieter; use --verbose if necessary (this message is temporary)
input tx index = 0; tx input vout = 0; value = 100000000
got witness stack of size 3
34 bytes (v0=P2WSH, v1=taproot/tapscript)
Taproot commitment:
- control  = c150929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661df4464b2d
- program  = ee97c425a556413759ff1dcc75e05fe6bc91890784346a1b44a88b470dbbf50c
- script   = 2023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b2
- path len = 2
- p        = 50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
- q        = ee97c425a556413759ff1dcc75e05fe6bc91890784346a1b44a88b470dbbf50c
- k        = 9836b89dc3bf2e25366f0f12cce5789c315f22db2681988115274f49596a3d00          (tap leaf hash)
  (TapLeaf(0xc0 || 2023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b2))
valid script
- generating prevout hash from 1 ins
[+] COutPoint(bd05e0c122, 0)
9 op script loaded. type `help` for usage information
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
<<< taproot commitment >>>                                         |                                                               i: 0
Branch: 339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd5... | k: 003d6a59494f271581988126db225f319c78e5cc120f6f36252ebfc39db8...
Branch: eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661... | 
Tweak: 50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9a... | 
CheckTapTweak                                                      | 
<<< committed script >>>                                           | 
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890   | 
OP_CHECKSIG                                                        | 
OP_VERIFY                                                          | 
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0000 Branch: 339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817
btcdeb> step
- looping over path (0..1)
  - 0: node = 33...; taproot control node match -> k first
  (TapBranch(TapLeaf(0xc0 || 2023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b2) || Span<33,32>=339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817))
  - 0: k -> 75cde686b351a948b0c04d6fa0cd1b69537f4bb78f01c746e2ad654c2cc2b87a
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
<<< taproot commitment >>>                                         |                                                               i: 1
Branch: 339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd5... | k: 7ab8c22c4c65ade246c7018fb74b7f53691bcda06f4dc0b048a951b386e6...
Branch: eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661... | 
Tweak: 50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9a... | 
CheckTapTweak                                                      | 
<<< committed script >>>                                           | 
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890   | 
OP_CHECKSIG                                                        | 
OP_VERIFY                                                          | 
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0001 Branch: eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661df4464b2d
btcdeb> step
- looping over path (0..1)
  - 1: node = eb...; taproot control node match -> k first
  (TapBranch(TapBranch(TapLeaf(0xc0 || 2023c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890ac690114b2) || Span<33,32>=339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd59c4eaa817) || Span<65,32>=eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661df4464b2d))
  - 1: k -> 0ebe9555fb4a44b7ac3d4c2d7170de13eb48b27c26a3a648ae41be46935550f6
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
<<< taproot commitment >>>                                         |                                                               i: 2
Branch: 339ba936f2ade62e286fb5ff033c119a7798e7bb711c2242dbe4dd5... | k: f650559346be41ae48a6a3267cb248eb13de70712d4c3dacb7444afb5595...
Branch: eba257c5252d6a467010fe3aeb83c386f8a84f04632678db72f1661... | 
Tweak: 50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9a... | 
CheckTapTweak                                                      | 
<<< committed script >>>                                           | 
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890   | 
OP_CHECKSIG                                                        | 
OP_VERIFY                                                          | 
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0002 Tweak: 50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
btcdeb> step
- looping over path (0..1)
- q.CheckTapTweak(p, k, 1) == success
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890   | 7e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a6...
OP_CHECKSIG                                                        | 
OP_VERIFY                                                          | 
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0003 CheckTapTweak
btcdeb> step
                <> PUSH stack 23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSIG                                                        |   23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
OP_VERIFY                                                          | 7e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a6...
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0004 23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
btcdeb> step
EvalChecksig() sigversion=3
Eval Checksig Tapscript
- sig must not be empty: ok
- validation weight - 50 -> 203
- 32 byte pubkey (new type); schnorr sig check
GenericTransactionSignatureChecker::CheckSchnorrSignature(64 len sig, 32 len pubkey, sigversion=3)
  sig         = 7e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a679c9634a0845e33e5ab4f9be93d31359499952b27ca26e08fe19651f7a7dd4aa4
  pub key     = 23c852dd9d1fa90d6d3bb0c4fff50f071e8e262492920b32267cec25a6770890
SignatureHashSchnorr(in_pos=0, hash_type=00)
- tapscript sighash
- schnorr sighash = 31ec0fa5ce0fc0c45ac04120e8c5bc0dd7cff13f3ae2c52a0ce24761bad0b274
  pubkey.VerifySchnorrSignature(sig=7e4a561693ea9676b703ffd5698b77132a1f5a594163a31e122f8bffac2b3a679c9634a0845e33e5ab4f9be93d31359499952b27ca26e08fe19651f7a7dd4aa4, sighash=31ec0fa5ce0fc0c45ac04120e8c5bc0dd7cff13f3ae2c52a0ce24761bad0b274):
  result: success
                <> POP  stack
                <> POP  stack
                <> PUSH stack 01
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_VERIFY                                                          |                                                                 01
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0005 OP_CHECKSIG
btcdeb> step
                <> POP  stack
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
14                                                                 | 
OP_CHECKSEQUENCEVERIFY                                             | 
#0006 OP_VERIFY
btcdeb> step
                <> PUSH stack 14
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSEQUENCEVERIFY                                             |                                                                 14
#0007 14
btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
                                                                   |                                                                 14
#0008 OP_CHECKSEQUENCEVERIFY
btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
                                                                   |                                                                 14
#0008 OP_CHECKSEQUENCEVERIFY
btcdeb> step
at end of script
```

### Slashing

```bash
btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
acb57de852c0372f871efc256e5feb2e3f9534fb2db433f3460cb9f85e07f707   | 5baff95d35ad142adfa6e49f178e082e57c9f8de56850924d0a182410a8656a...
OP_CHECKSIG                                                        |                                                                 ''
OP_VERIFY                                                          | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
fe151a768cd8c48a2a27551a4f6d37916b8714adcdb9c2f1137f294dfcf69d9e   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIG                                                        |
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   |
OP_CHECKSIGADD                                                     |
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSIG                                                        |   acb57de852c0372f871efc256e5feb2e3f9534fb2db433f3460cb9f85e07f707
OP_VERIFY                                                          | 5baff95d35ad142adfa6e49f178e082e57c9f8de56850924d0a182410a8656a...
fe151a768cd8c48a2a27551a4f6d37916b8714adcdb9c2f1137f294dfcf69d9e   |                                                                 ''
OP_CHECKSIG                                                        | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIGADD                                                     |
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_VERIFY                                                          |                                                                 01
fe151a768cd8c48a2a27551a4f6d37916b8714adcdb9c2f1137f294dfcf69d9e   |                                                                 ''
OP_CHECKSIG                                                        | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIGADD                                                     |
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
fe151a768cd8c48a2a27551a4f6d37916b8714adcdb9c2f1137f294dfcf69d9e   |                                                                 ''
OP_CHECKSIG                                                        | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIGADD                                                     |
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSIG                                                        |   fe151a768cd8c48a2a27551a4f6d37916b8714adcdb9c2f1137f294dfcf69d9e
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   |                                                                 ''
OP_CHECKSIGADD                                                     | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f   |                                                                 00
OP_CHECKSIGADD                                                     | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_CHECKSIGADD                                                     |
02                                                                 |
OP_NUMEQUAL                                                        |

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSIGADD                                                     |   db90535a94f32cf07785cd6aac194780df26e2c2caf17340faef13f91864f58f
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |                                                                 00
OP_CHECKSIGADD                                                     | 50f34a830c2e713dd9e8e7c80f2458c4190af91c5f877c01efcf91b4bd0f1d2...
02                                                                 | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
OP_NUMEQUAL                                                        | 

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd   |                                                                 01
OP_CHECKSIGADD                                                     | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...
02                                                                 | 
OP_NUMEQUAL                                                        | 

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_CHECKSIGADD                                                     |   54f665f1255d355867bcdd4dca48c554980738f9868ab713a5a894449a7f3edd
02                                                                 |                                                                 01
OP_NUMEQUAL                                                        | c56832542c882b9a79abe7e76927c465e407e4f18ee34aa14a6ead0e050e2e0...

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
02                                                                 |                                                                 03
OP_NUMEQUAL                                                        | 

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
OP_NUMEQUAL                                                        |                                                                 03
                                                                   |                                                                 03

btcdeb> step
script                                                             |                                                             stack 
-------------------------------------------------------------------+-------------------------------------------------------------------
                                                                   |                                                                 01
```
